# Example code of Diffusion
Colab 환경에서 Diffusion 모델을 이미지를 생성해보는 예제입니다.

## Colab 환경 설정
예제를 실행시키기 위해 python package들을 설치합니다. 예제로 사용할 이미지들도 다운로드 받습니다.

In [ ]:
# Local에서 Run하는 경우 False로 변경
using_colab = True

In [ ]:
if using_colab:
    !wget https://raw.githubusercontent.com/mrsyee/dl_apps/main/image_generation/requirements-colab.txt
    !pip install -r requirements-colab.txt

    # Download examples
    !mkdir examples
    !cd examples && wget https://raw.githubusercontent.com/mrsyee/dl_apps/main/image_generation/examples/kitty.png

## Import dependency

In [ ]:
import torch
from diffusers import EulerAncestralDiscreteScheduler, StableDiffusionImg2ImgPipeline
from diffusers.utils import make_image_grid
from PIL import Image

## Initialize Model

In [ ]:
pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
).to("cuda")

## 예제 이미지 불러오기

In [ ]:
input_image = Image.open("examples/kitty.png")
make_image_grid([input_image], rows=1, cols=1)

## Inference pipline

In [ ]:
prompt = (
    "kitty in the city, cartoon style, drawing, detailed"
)
negative_prompt = "ugly, blurry, bad anatomy, bad art, wierd colors"

In [ ]:
output_images = pipeline(
    image=input_image,
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_images_per_prompt=4,
    num_inference_steps=30,
    strength=0.7,
).images

In [ ]:
make_image_grid(output_images, rows=1, cols=4)

## 다양한 diffusion 모델 다운로드하기
- [CivitAI](https://civitai.com/models)

In [ ]:
import os

import requests
from tqdm import tqdm

In [ ]:
url = "https://civitai.com/models/65203/disney-pixar-cartoon-type-a"

In [ ]:
model_id = url.replace("https://civitai.com/models/", "").split("/")[0]
model_id

In [ ]:
response = requests.get(f"https://civitai.com/api/v1/models/{model_id}")
response.json()

In [ ]:
download_url = response.json()["modelVersions"][0]["downloadUrl"]
filename = response.json()["modelVersions"][0]["files"][0]["name"]

In [ ]:
print(download_url)
print(filename)

In [ ]:
def download_from_url(url: str, file_path: str, chunk_size=1024):
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    with open(file_path, 'wb') as file, tqdm(
        desc=file_path,
        total=total,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
            size = file.write(data)
            bar.update(size)

In [ ]:
file_path = f"models/{filename}"
os.makedirs("models", exist_ok=True)
print(f"[INFO] Download start!")
download_from_url(download_url, file_path)
print(f"\n[INFO] File downloaded: {file_path}")

## 다운로드 받은 모델로 이미지 생성하기

In [ ]:
pipeline = StableDiffusionImg2ImgPipeline.from_single_file(
        "models/disneyPixarCartoon_v10.safetensors",
        torch_dtype=torch.float16,
        use_safetensors=True,
).to("cuda")

In [ ]:
input_image = Image.open("examples/kitty.png")

prompt = (
    "kitty in the city, cartoon style, drawing, detailed"
)
negative_prompt = "ugly, blurry, bad anatomy, bad art, wierd colors"

output_images = pipeline(
    image=input_image,
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_images_per_prompt=4,
    num_inference_steps=30,
    strength=0.7,
).images

In [ ]:
make_image_grid(output_images, rows=1, cols=4)